In [98]:
import numpy as np
import teradata
import pandas as pd
from pandas import Series,DataFrame

import math

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

from sklearn import metrics
import statsmodels.api as sm
from sklearn import preprocessing
import glob
import datetime
import csv

In [135]:
path_archive =r'/Users/yuanyao/Documents/underwriting/archive/' # use your path

In [136]:
path =r'/Users/yuanyao/Documents/underwriting/underwriting/' # use your path
allFiles = glob.glob(path + "/*.txt")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,sep=",",dtype=object,index_col=None, header=None)
    list_.append(df)
frame = pd.concat(list_)

In [137]:
hold_path =r'/Users/yuanyao/Documents/underwriting/underwriting_hold/' # use your path
allFiles = glob.glob(hold_path + "/*.txt")
hold_frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,sep=",",dtype=object,index_col=None, header=None)
    list_.append(df)
hold_frame = pd.concat(list_)

In [103]:
sql_query=""" select * from pp_oap_yaoy_t.uw_recent_dt ;"""
uw_recent_dt=pd.read_sql(sql_query,simba)

In [104]:
sql_query2="""SEL base.*,cust.busn_name
,a.score, a.auto_tier, c.most_recent_rvw_dt
FROM pp_oap_seller_lm_t.sc_uw_hold_leads_AM base
left join  pp_discovery_views.dim_customer cust on  base.cust_id = cust.cust_id
LEFT  JOIN pp_oap_seller_lm_t.sc_uw_scr_auto_tier a ON base.cust_id = a.cust_id
LEFT JOIN pp_oap_yaoy_t.uw_recent_dt c on base.cust_id = c.cust_id
where  base.AM_NAME not in ('SB', 'Regional AM')
and base.cust_id not in (
SEL pp_acct_id_notes FROM pp_oap_seller_lm_t.sc_uw_hold_leads_AM base2
LEFT JOIN pp_access_views.dw_sf_case d ON base2.cust_id = d.pp_acct_id_notes
where d.case_sub LIKE ('%periodic risk review%')
AND d.acct_cntry IN ('US', 'CA')
AND d.pp_acct_id_notes IS NOT NULL
)
order by score;
"""
uw_no_am=pd.read_sql(sql_query2,simba)

In [138]:
# the am UW send
uw_score=uw_no_am.loc[uw_no_am['score']<740]
uw_am=uw_score.loc[~uw_score['cust_id'].isin(hold_frame[1])]
uw_am['GRP']='Organic Underwriting'
uw_am_send=uw_am[['cust_id','busn_name','GRP','AM_NAME','AM_SEGMENT','reasons']]

In [106]:
sql_query_pram="""SEL 
base.cust_id, base.reason as reasons
,cust.busn_name
, 'Periodic Review' as grp
, base.am_name
, base.am_segment
, date -1 as cutoff_dt
, base.score
, ' ' as auto_tier
, base.most_recent_rvw_dt
, 'PR' as seg
, date+14 as send_dt
FROM pp_oap_yaoy_t.pr_routing base
left join  pp_discovery_views.dim_customer cust on  base.cust_id = cust.cust_id
where base.am_name not in ('BSS','?', 'Channel Partner', 'Regional AM','Regional AM')
and base.am_segment not in ('BSS','?', 'Channel Partner', 'Regional AM','Regional AM')
and base.most_recent_rvw_dt<= current_date-180
ORDER BY base.score;
"""
pr_am=pd.read_sql(sql_query_pram,simba)

In [139]:
# the AM PR send
pr_am_score=pr_am.loc[pr_am['score']<790]
pr_am_score.shape
pr_am_dedup=pr_am_score.loc[~pr_am_score['cust_id'].isin(hold_frame[1])]
pr_am_dedup=pr_am_dedup.loc[~pr_am_dedup['cust_id'].isin(frame[1])]
pr_am_dedup['GRP']='Periodic Review'
pr_am_sent=pr_am_dedup[['cust_id','busn_name','GRP','AM_NAME','AM_SEGMENT','reasons']]
pr_am_sent=pr_am_sent.drop_duplicates(subset=['cust_id'], keep="last")

In [146]:
sql_query_pram_more="""
SEL 
base.cust_id
,'04. Past Due' as reasons
, 'Periodic Review' as grp
,cust.busn_name
,mm.AM_NAME_2017 AS AM_NAME
,mm.AM_SEGMENT_2017 AS AM_SEGMENT
,date -1 as cutoff_dt
,base.score
, BASE.auto_tier
,b.most_recent_rvw_dt
,'PR' as seg
,date +14 as send_dt
--, base.LMS_flag
FROM pp_oap_seller_lm_t.cl_pr_freq_new base
LEFT OUTER JOIN pp_oap_seller_lm_t.as_mm_segments_2017 mm ON base.cust_id = mm.cust_id
Left join pp_oap_yaoy_t.uw_recent_dt b on base.cust_id = b.cust_id
left join  pp_discovery_views.dim_customer cust on  base.cust_id = cust.cust_id
where b.most_recent_rvw_dt <= date -180
-- AM_accounts
and 
mm.am_segment_2017 is not null and mm.am_segment_2017 not in ('BSS', 'CP','Regional AM','NA MOR AM','Regional AM')
and mm.AM_NAME_2017 not in ('SB', 'Regional AM')
-- Non_am
--and (mm.am_segment_2017 is null or mm.am_segment_2017 in ('BSS','?', 'CP'))
--and base.LMS_flag_date >= '2016-12-01' 
AND base.score IS NOT NULL
ORDER BY base.score;
"""
pr_am_more=pd.read_sql(sql_query_pram_more,simba)

In [147]:
# the am PR send more
pr_am_dedup_more=pr_am_more.loc[~pr_am_more['cust_id'].isin(hold_frame[1])]
pr_am_dedup_more=pr_am_dedup_more.loc[~pr_am_dedup_more['cust_id'].isin(frame[1])]
pr_am_dedup_more=pr_am_dedup_more.loc[pr_am_dedup_more['score']<850]
pr_am_dedup_more['GRP']='Periodic Review'
pr_am_sent_more=pr_am_dedup_more[['cust_id','busn_name','GRP','AM_NAME','AM_SEGMENT','reasons']]
pr_am_sent_more=pr_am_sent_more.drop_duplicates(subset=['cust_id'], keep="last")

In [148]:
# prepare the deduped archive dataset
am_send=pd.concat([uw_am_send, pr_am_sent,pr_am_sent_more], ignore_index=True)
am_send_dedup=am_send.drop_duplicates(subset=['cust_id'], keep="last")

In [149]:
am_send_dedup

,cust_id,busn_name,GRP,AM_NAME,AM_SEGMENT,reasons
0,1547285329960505920,Nakfour Trading LLC,Organic Underwriting,Tony Ward,Group 2-AM,07 UW Model ...
1,2220041088093664248,Upward Mobile,Organic Underwriting,Tony Ward,Group 2-AM,07 UW Model ...
2,2234180943456572959,Silicon Andhra,Organic Underwriting,Kathleen Gotto,Group 2-AM,07 PRM Score ...
3,1503917583007047378,Country Cat INC,Organic Underwriting,Trevor Stoehr,Group 1-AM,07 PRM Score ...
4,1703816592470427738,Arrowhead,Organic Underwriting,Warren Gurnon,Group 1-AM,07 UW Model ...
5,1532948079835874175,BeenVerified Inc.,Organic Underwriting,Tiffany Yocum,Group 2-AM,07 UW Model ...
6,1820634767105371552,PoolSupply,Organic Underwriting,Rick Howell,Group 1-AM,07 UW Model ...
7,1999566003669031312,Veterinary Emergency & Critical Care Society,Organic Underwriting,Madoka Marshall,Group 2-AM,07 PRM Score ...
8,1726490563910283435,"Light O Rama, Inc.",Organic Underwriting,Ann Powers,Group 2-AM,07 UW Model ...
9,1162458580982477022,Cogswell Polytechnical College,Organic Underwriting,Trevor Stoehr,Group 1-AM,07 UW Model ...


In [150]:
# archive the am send including UW and PR
send_date=datetime.date.today()+datetime.timedelta(14)
am_send_dedup['send_dt']=send_date.strftime('%Y%m%d')
df_obj = am_send_dedup.select_dtypes(['object'])
am_send_dedup[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [151]:
am_send_dedup=am_send_dedup.loc[~am_send_dedup['cust_id'].isin(hold_frame[1])]
am_send_dedup=am_send_dedup.loc[~am_send_dedup['cust_id'].isin(frame[1])]

In [152]:
# send out as email
am_send_dedup

,cust_id,busn_name,GRP,AM_NAME,AM_SEGMENT,reasons,send_dt
0,1547285329960505920,Nakfour Trading LLC,Organic Underwriting,Tony Ward,Group 2-AM,07 UW Model,20171113
1,2220041088093664248,Upward Mobile,Organic Underwriting,Tony Ward,Group 2-AM,07 UW Model,20171113
2,2234180943456572959,Silicon Andhra,Organic Underwriting,Kathleen Gotto,Group 2-AM,07 PRM Score,20171113
3,1503917583007047378,Country Cat INC,Organic Underwriting,Trevor Stoehr,Group 1-AM,07 PRM Score,20171113
4,1703816592470427738,Arrowhead,Organic Underwriting,Warren Gurnon,Group 1-AM,07 UW Model,20171113
5,1532948079835874175,BeenVerified Inc.,Organic Underwriting,Tiffany Yocum,Group 2-AM,07 UW Model,20171113
6,1820634767105371552,PoolSupply,Organic Underwriting,Rick Howell,Group 1-AM,07 UW Model,20171113
7,1999566003669031312,Veterinary Emergency & Critical Care Society,Organic Underwriting,Madoka Marshall,Group 2-AM,07 PRM Score,20171113
8,1726490563910283435,"Light O Rama, Inc.",Organic Underwriting,Ann Powers,Group 2-AM,07 UW Model,20171113
9,1162458580982477022,Cogswell Polytechnical College,Organic Underwriting,Trevor Stoehr,Group 1-AM,07 UW Model,20171113


In [153]:
# put into txt file
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_archive='zoot_all_send'+ today_path +'_hold.txt'
am_send_dedup.to_csv(path_archive+filename_archive, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

In [154]:
# prepare the am send file
am_send_ow=am_send_dedup[am_send_dedup['GRP']=='Organic Underwriting']
am_send_ow['upd_Referral_Codes']='ORGUWD'
am_send_ow['Account_Number']=am_send_ow['cust_id']
send_date=datetime.date.today()+datetime.timedelta(14)
am_send_ow['Referral_Date']=send_date.strftime('%Y%m%d')
am_send_ow['_comment']=am_send_ow['reasons']
am_send_ow['_routing_reason_codes']=np.nan
am_send_ow['Priority_Score']=750
am_send_ow['Detections_Exposure']=0
am_send_ow['CAP_Value']=0
am_send_ow['EBay_AVERAGE_Pricing']=0
am_send_ow['_industry']=np.nan
am_send_ow['_merchant_type_long']="D2S"
am_send_ow['Thirty_Day_Rolling_TPV']=0
am_send_ow['Shadow_Monit_Flag']=0
am_send_ow['Account_Managed_Flag']=0
df_obj = am_send_ow.select_dtypes(['object'])
am_send_ow[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
am_send_csv=am_send_ow[['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']]
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_ow='zoot_ouw_referral_NA_'+ today_path +'_hold.txt'
filename_ow
am_send_csv.to_csv(hold_path+filename_ow, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [155]:
# prepare the file for am PR
am_send_pr=am_send_dedup[am_send_dedup['GRP']=='Periodic Review']
am_send_pr['upd_Referral_Codes']='PERIODICREV'
am_send_pr['Account_Number']=am_send_pr['cust_id']
send_date=datetime.date.today()+datetime.timedelta(14)
am_send_pr['Referral_Date']=send_date.strftime('%Y%m%d')
am_send_pr['_comment']=am_send_pr['reasons']
am_send_pr['_routing_reason_codes']=np.nan
am_send_pr['Priority_Score']=750
am_send_pr['Detections_Exposure']=0
am_send_pr['CAP_Value']=0
am_send_pr['EBay_AVERAGE_Pricing']=0
am_send_pr['_industry']=np.nan
am_send_pr['_merchant_type_long']="D2S"
am_send_pr['Thirty_Day_Rolling_TPV']=0
am_send_pr['Shadow_Monit_Flag']=0
am_send_pr['Account_Managed_Flag']=0
df_obj = am_send_pr.select_dtypes(['object'])
am_send_pr[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
am_send_pr_csv=am_send_pr[['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']]
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_pr='zoot_PR_referral_NA_'+ today_path +'_hold.txt'
am_send_pr_csv.to_csv(hold_path+filename_pr, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [156]:
# start the non-am UW
sql_uw_nonam="""SEL base.* 
,b.score, b.auto_tier, c.most_recent_rvw_dt
FROM pp_oap_seller_lm_t.sc_uw_hold_leads_non_AM base
LEFT OUTER JOIN pp_oap_seller_lm_t.sc_uw_scr_auto_tier b ON base.cust_id = b.cust_id
LEFT JOIN pp_oap_yaoy_t.uw_recent_dt c on base.cust_id = c.cust_id
union all
SEL base.*
,a.score, a.auto_tier, c.most_recent_rvw_dt
FROM pp_oap_seller_lm_t.sc_uw_hold_leads_AM base
LEFT OUTER JOIN pp_oap_seller_lm_t.sc_uw_scr_auto_tier a ON base.cust_id = a.cust_id
LEFT JOIN pp_oap_yaoy_t.uw_recent_dt c on base.cust_id = c.cust_id
AND base.AM_NAME in ('SB', 'Channel Partner')
and base.cust_id not in (
SEL pp_acct_id_notes FROM pp_oap_seller_lm_t.sc_uw_hold_leads_AM base2
LEFT JOIN pp_access_views.dw_sf_case d ON base2.cust_id = d.pp_acct_id_notes
where d.case_sub LIKE ('%periodic risk review%')
AND d.acct_cntry IN ('US', 'CA')
AND d.pp_acct_id_notes IS NOT NULL
);
"""

In [176]:
ow_nonam=pd.read_sql(sql_uw_nonam,simba)

In [177]:
uw_noam=ow_nonam.loc[~ow_nonam['cust_id'].isin(frame[1])]

In [178]:
uw_noam_score=uw_noam.loc[uw_noam['score']<690]

In [179]:
uw_noam_dedup=uw_noam_score.loc[~uw_noam_score['cust_id'].isin(frame[1])]
uw_noam_dedup['GRP']='Organic Underwriting'
uw_noam_sent=uw_noam_dedup[['cust_id','GRP','AM_NAME','AM_SEGMENT','reasons','most_recent_rvw_dt','Auto_Tier']]
uw_noam_sent=uw_noam_sent.drop_duplicates(subset=['cust_id'], keep="last")

In [180]:
uw_noam_sent

,cust_id,GRP,AM_NAME,AM_SEGMENT,reasons,most_recent_rvw_dt,Auto_Tier
21,1547285329960505920,Organic Underwriting,Tony Ward,Group 2-AM,07 UW Model ...,None,Auto Tier 5
40,2121196980818728060,Organic Underwriting,None,None,03 ASP Change ...,None,Auto Tier 4
41,2122127844081002000,Organic Underwriting,None,None,06 High TPV Spike ...,None,Auto Tier 4
43,2220041088093664248,Organic Underwriting,Tony Ward,Group 2-AM,07 UW Model ...,None,Auto Tier 4
47,2234180943456572959,Organic Underwriting,Kathleen Gotto,Group 2-AM,07 PRM Score ...,None,Auto Tier 3


In [181]:
# start the non-am pr
sql_pr_nonam="""SEL 
base.cust_id, base.reason as reasons
, 'Periodic Review' as grp
, base.am_name
, base.am_segment
, date as cutoff_dt
, base.score
, ' ' as Auto_Tier
, base.most_recent_rvw_dt
, 'PR' as seg
, date as send_dt
FROM pp_oap_yaoy_t.pr_routing base
WHERE (base.most_recent_rvw_dt <= date -180 or base.most_recent_rvw_dt is null)
AND (base.am_name in ('BSS','?', 'Channel Partner', 'Regional AM','LE') OR base.am_name IS NULL)
And base.most_recent_rvw_dt<= current_date-210
ORDER BY base.score asc
"""

In [182]:
pr_nonam=pd.read_sql(sql_pr_nonam,simba)
pr_noam_dedup=pr_nonam.loc[~pr_nonam['cust_id'].isin(frame[1])]
pr_noam_score=pr_noam_dedup.loc[pr_noam_dedup['score']<720]
pr_noam_dedup=pr_noam_score.loc[~pr_noam_score['cust_id'].isin(frame[1])]
pr_noam_dedup['GRP']='Periodic Review'
pr_noam_sent=pr_noam_dedup[['cust_id','GRP','AM_NAME','AM_SEGMENT','reasons','most_recent_rvw_dt','Auto_Tier']]
pr_noam_sent=pr_noam_sent.drop_duplicates(subset=['cust_id'], keep="last")
pr_noam_sent

,cust_id,GRP,AM_NAME,AM_SEGMENT,reasons,most_recent_rvw_dt,Auto_Tier
0,2104050881540723254,Periodic Review,BSS,BSS,Over Limit,2017-03-28,
1,1974734765928739407,Periodic Review,BSS,BSS,Auto Tier,2017-04-03,
2,1617361284116684474,Periodic Review,None,None,Over Limit,2014-08-13,
3,1626270144316121027,Periodic Review,BSS,BSS,Auto Tier,2017-03-14,
4,1891270999530639403,Periodic Review,BSS,BSS,Over Limit,2017-03-28,
5,1755731456602294190,Periodic Review,BSS,BSS,Over Limit,2017-03-14,
6,2134880999358048933,Periodic Review,BSS,BSS,Over Limit,2017-03-30,
7,2302659338743701145,Periodic Review,BSS,BSS,Past Due - Org,2017-03-13,
8,1377961451477889014,Periodic Review,BSS,BSS,Over Limit,2017-03-08,
9,2132394689710764575,Periodic Review,BSS,BSS,Over Limit,2017-03-14,


In [183]:
# start the non-am UW more
sql_pr_nonam_more="""SEL 
base.cust_id
,'07. Past Due' as reasons
, 'Periodic Review' as grp
,mm.AM_NAME_2017 AS AM_NAME
,mm.AM_SEGMENT_2017 AS AM_SEGMENT
,date -1 as cutoff_dt
,base.score
, BASE.Auto_Tier
,b.most_recent_rvw_dt
,'PR' as seg
,date +14 as send_dt
--, base.LMS_flag
FROM pp_oap_seller_lm_t.cl_pr_freq_new base
LEFT  JOIN pp_oap_seller_lm_t.as_mm_segments_2017 mm ON base.cust_id = mm.cust_id
Left join pp_oap_yaoy_t.uw_recent_dt b on base.cust_id = b.cust_id
where b.most_recent_rvw_dt <= date -180
-- Non_am
and (mm.am_segment_2017 is null or mm.am_segment_2017 in ('BSS','?', 'CP'))
--and base.LMS_flag_date >= '2016-12-01' 
AND base.score IS NOT NULL
ORDER BY base.score;
"""

In [184]:
pr_nonam_more=pd.read_sql(sql_pr_nonam_more,simba)
pr_noam_more_dedup=pr_nonam_more.loc[~pr_nonam_more['cust_id'].isin(frame[1])]
pr_noam_more_score=pr_noam_more_dedup.loc[pr_noam_more_dedup['score']<720]
pr_noam_more_dedup=pr_noam_more_score.loc[~pr_noam_more_score['cust_id'].isin(frame[1])]
pr_noam_more_dedup['GRP']='Periodic Review'
pr_noam_more_sent=pr_noam_more_dedup[['cust_id','GRP','AM_NAME','AM_SEGMENT','reasons','most_recent_rvw_dt','Auto_Tier']]
pr_noam_more_sent=pr_noam_more_sent.drop_duplicates(subset=['cust_id'], keep="last")

In [185]:
pr_noam_more_sent

,cust_id,GRP,AM_NAME,AM_SEGMENT,reasons,most_recent_rvw_dt,Auto_Tier


In [186]:
# prepare the deduped archive dataset
nonam_send=pd.concat([uw_noam_sent, pr_noam_sent,pr_noam_more_sent], ignore_index=True)
noam_send_dedup=nonam_send.drop_duplicates(subset=['cust_id'], keep="last")
noam_send_dedup=noam_send_dedup.loc[~noam_send_dedup['cust_id'].isin(frame[1])]

In [187]:
noam_send_dedup

,cust_id,GRP,AM_NAME,AM_SEGMENT,reasons,most_recent_rvw_dt,Auto_Tier
0,1547285329960505920,Organic Underwriting,Tony Ward,Group 2-AM,07 UW Model ...,None,Auto Tier 5
1,2121196980818728060,Organic Underwriting,None,None,03 ASP Change ...,None,Auto Tier 4
2,2122127844081002000,Organic Underwriting,None,None,06 High TPV Spike ...,None,Auto Tier 4
3,2220041088093664248,Organic Underwriting,Tony Ward,Group 2-AM,07 UW Model ...,None,Auto Tier 4
4,2234180943456572959,Organic Underwriting,Kathleen Gotto,Group 2-AM,07 PRM Score ...,None,Auto Tier 3
5,2104050881540723254,Periodic Review,BSS,BSS,Over Limit,2017-03-28,
6,1974734765928739407,Periodic Review,BSS,BSS,Auto Tier,2017-04-03,
7,1617361284116684474,Periodic Review,None,None,Over Limit,2014-08-13,
8,1626270144316121027,Periodic Review,BSS,BSS,Auto Tier,2017-03-14,
9,1891270999530639403,Periodic Review,BSS,BSS,Over Limit,2017-03-28,


In [188]:
# archive the noam send including UW and PR
send_date=datetime.date.today()+datetime.timedelta(14)
noam_send_dedup['send_dt']=send_date.strftime('%Y%m%d')
df_obj = noam_send_dedup.select_dtypes(['object'])
noam_send_dedup[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_archive='zoot_all_send'+ today_path +'.txt'
noam_send_dedup.to_csv(path_archive+filename_archive, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

In [192]:
# prepare the UW nonam send
noam_send_uw_final=noam_send_dedup[noam_send_dedup['GRP']=='Organic Underwriting']
noam_send_uw_final['upd_Referral_Codes']='ORGUWD'
noam_send_uw_final['Account_Number']=noam_send_uw_final['cust_id']
noam_send_uw_final['Referral_Date']=pd.Timestamp('today').strftime('%Y%m%d')
noam_send_uw_final['_comment']=noam_send_uw_final['reasons']
noam_send_uw_final['_routing_reason_codes']=np.nan
noam_send_uw_final['Priority_Score']=730
noam_send_uw_final['Detections_Exposure']=0
noam_send_uw_final['CAP_Value']=0
noam_send_uw_final['EBay_AVERAGE_Pricing']=0
noam_send_uw_final['_industry']=np.nan
noam_send_uw_final['_merchant_type_long']="D2S"
noam_send_uw_final['Thirty_Day_Rolling_TPV']=0
noam_send_uw_final['Shadow_Monit_Flag']=0
noam_send_uw_final['Account_Managed_Flag']=0
df_obj = noam_send_uw_final.select_dtypes(['object'])
noam_send_uw_final[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
noam_send_uw_csv=noam_send_uw_final[['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']]
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_ow='zoot_ouw_referral_NA_'+ today_path +'.txt'
noam_send_uw_csv.to_csv(path+filename_ow, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [190]:
noam_send_uw_csv

,upd_Referral_Codes,Account_Number,Referral_Date,_comment,_routing_reason_codes,Priority_Score,Detections_Exposure,CAP_Value,EBay_AVERAGE_Pricing,_industry,_merchant_type_long,Thirty_Day_Rolling_TPV,Shadow_Monit_Flag,Account_Managed_Flag
0,ORGUWD,1547285329960505920,20171030,07 UW Model,NaN,730,0,0,0,NaN,D2S,0,0,0
1,ORGUWD,2121196980818728060,20171030,03 ASP Change,NaN,730,0,0,0,NaN,D2S,0,0,0
2,ORGUWD,2122127844081002000,20171030,06 High TPV Spike,NaN,730,0,0,0,NaN,D2S,0,0,0
3,ORGUWD,2220041088093664248,20171030,07 UW Model,NaN,730,0,0,0,NaN,D2S,0,0,0
4,ORGUWD,2234180943456572959,20171030,07 PRM Score,NaN,730,0,0,0,NaN,D2S,0,0,0


In [191]:
# prepare the pr nonam send
noam_send_pr_final=noam_send_dedup[noam_send_dedup['GRP']=='Periodic Review']
noam_send_pr_final['upd_Referral_Codes']='PERIODICREV'
noam_send_pr_final['Account_Number']=noam_send_pr_final['cust_id']
noam_send_pr_final['Referral_Date']=pd.Timestamp('today').strftime('%Y%m%d')
noam_send_pr_final['_comment']=noam_send_pr_final['reasons']
noam_send_pr_final['_routing_reason_codes']=np.nan
noam_send_pr_final['Priority_Score']=720
noam_send_pr_final['Detections_Exposure']=0
noam_send_pr_final['CAP_Value']=0
noam_send_pr_final['EBay_AVERAGE_Pricing']=0
noam_send_pr_final['_industry']=np.nan
noam_send_pr_final['_merchant_type_long']="D2S"
noam_send_pr_final['Thirty_Day_Rolling_TPV']=0
noam_send_pr_final['Shadow_Monit_Flag']=0
noam_send_pr_final['Account_Managed_Flag']=0
df_obj = noam_send_pr_final.select_dtypes(['object'])
noam_send_pr_final[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
noam_send_pr_csv=noam_send_pr_final[['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']]
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_pr='zoot_PR_referral_NA_'+ today_path +'.txt'
noam_send_pr_csv.to_csv(path+filename_pr, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [173]:
noam_send_pr_csv

,upd_Referral_Codes,Account_Number,Referral_Date,_comment,_routing_reason_codes,Priority_Score,Detections_Exposure,CAP_Value,EBay_AVERAGE_Pricing,_industry,_merchant_type_long,Thirty_Day_Rolling_TPV,Shadow_Monit_Flag,Account_Managed_Flag
5,PERIODICREV,2104050881540723254,20171030,Over Limit,NaN,720,0,0,0,NaN,D2S,0,0,0
6,PERIODICREV,1974734765928739407,20171030,Auto Tier,NaN,720,0,0,0,NaN,D2S,0,0,0
7,PERIODICREV,1617361284116684474,20171030,Over Limit,NaN,720,0,0,0,NaN,D2S,0,0,0
8,PERIODICREV,1626270144316121027,20171030,Auto Tier,NaN,720,0,0,0,NaN,D2S,0,0,0
9,PERIODICREV,1891270999530639403,20171030,Over Limit,NaN,720,0,0,0,NaN,D2S,0,0,0
10,PERIODICREV,1755731456602294190,20171030,Over Limit,NaN,720,0,0,0,NaN,D2S,0,0,0
11,PERIODICREV,2134880999358048933,20171030,Over Limit,NaN,720,0,0,0,NaN,D2S,0,0,0
12,PERIODICREV,2302659338743701145,20171030,Past Due - Org,NaN,720,0,0,0,NaN,D2S,0,0,0
13,PERIODICREV,1377961451477889014,20171030,Over Limit,NaN,720,0,0,0,NaN,D2S,0,0,0
14,PERIODICREV,2132394689710764575,20171030,Over Limit,NaN,720,0,0,0,NaN,D2S,0,0,0


In [174]:
# prepare the pr nonam send change with combine
noam_send_pr_final=noam_send_dedup[noam_send_dedup['GRP']=='Periodic Review']
noam_send_pr_final['upd_Referral_Codes']='PERIODICREV'
noam_send_pr_final['Account_Number']=noam_send_pr_final['cust_id']
noam_send_pr_final['Referral_Date']=pd.Timestamp('today').strftime('%Y%m%d')
noam_send_pr_final['_comment']=noam_send_pr_final['reasons']
noam_send_pr_final['_routing_reason_codes']=np.nan
noam_send_pr_final['Priority_Score']=750
noam_send_pr_final['Detections_Exposure']=0
noam_send_pr_final['CAP_Value']=0
noam_send_pr_final['EBay_AVERAGE_Pricing']=0
noam_send_pr_final['_industry']=np.nan
noam_send_pr_final['_merchant_type_long']="D2S"
noam_send_pr_final['Thirty_Day_Rolling_TPV']=0
noam_send_pr_final['Shadow_Monit_Flag']=0
noam_send_pr_final['Account_Managed_Flag']=0
noam_send_pr_csv=noam_send_pr_final[['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']]
# attach hold file
cols=['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']
pre_pr_hold = pd.read_csv(hold_path+'zoot_PR_referral_NA_20170807_hold.txt', names=cols,header=None)
pr_send_all=pd.concat([pre_pr_hold,noam_send_pr_csv], ignore_index=True)
# strip column
df_obj = pr_send_all.select_dtypes(['object'])
pr_send_all[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_pr='zoot_PR_referral_NA_'+ today_path +'.txt'
# craete send file
pr_send_all.to_csv(path+filename_pr, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

UnicodeEncodeError: 'ascii' codec can't encode character u'\u201d' in position 21: ordinal not in range(128)

In [175]:
pr_send_all=pd.concat([pre_pr_hold,noam_send_pr_csv], ignore_index=True)
pr_send_all.to_csv(path+filename_ow, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
# craete send file
pr_send_all.to_csv(path+filename_ow, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
cols=['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']
pre_pr_hold = pd.read_csv(hold_path+'zoot_PR_referral_NA_20170807_hold.txt', names=cols,header=None)

In [ ]:
pr_send_all

In [ ]:
noam_send_pr_csv

In [ ]:
ow_merged = pre_ouw.merge(pre_ouw_hold,on=label)